# Downloading dataset by dataset


# 1. Dataset

## 1.1 Upload dataset files from HuggingFace

### BESSTIE-reddit-sentiment-uk/

In [1]:
import pandas as pd

splits = {'train': 'reddit-sentiment-uk-train.jsonl', 'validation': 'reddit-sentiment-uk-valid.jsonl'}
df_reddit_sentiment_uk = pd.read_json("hf://datasets/mindhunter23/BESSTIE-reddit-sentiment-uk/" + splits["train"], lines=True)
df_reddit_sentiment_uk

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,id,text,sentiment_label
0,1cimjpr,"So instead of making savings, they continued t...",0
1,1d35qlg,Needless story to have dragged into the electi...,0
2,1d3i3mt,"Now, in an ideal world there would be insight ...",0
3,1d5a8wa,How did you not get mind controlled at birth t...,0
4,1d5l3e9,"Talk lately of conscription, having a store of...",0
...,...,...,...
1002,1b5iodf,How is this a non-story? A shop will bow to th...,0
1003,1cv6iym,"The smoke screen is real, and Suella (along wi...",0
1004,1c9kjse,It's a really serious problem that young white...,0
1005,1b22zae,Good luck. Imagine if you refuse to see a PA b...,0


In [18]:
from our_preprocessing import Stemmer, text_preprocess, class_distribution

print('Original:\n', df_reddit_sentiment_uk.loc[0].text,'\n')
print('Lemma. with stopwords:\n',text_preprocess(df_reddit_sentiment_uk.loc[0].text, remove_stopwords=False, stemmer=Stemmer.WordNet), '\n')
print('Lemma. without stopwords:\n',text_preprocess(df_reddit_sentiment_uk.loc[0].text, remove_stopwords=True, stemmer=Stemmer.WordNet),'\n')
print('Stemmer with stopwords:\n',text_preprocess(df_reddit_sentiment_uk.loc[0].text, remove_stopwords=False),'\n')
print('Stemmer without stopwords:\n', text_preprocess(df_reddit_sentiment_uk.loc[0].text, remove_stopwords=True),'\n')

Original:
 So instead of making savings, they continued to spend money they didn’t have, yes that sounds very responsible. Maybe if the government had continued spending, the whole country would be in the same financial mess Birmingham is in. 

Lemma. with stopwords:
 ['so', 'instead', 'of', 'making', 'saving', 'they', 'continued', 'to', 'spend', 'money', 'they', 'didnt', 'have', 'yes', 'that', 'sound', 'very', 'responsible', 'maybe', 'if', 'the', 'government', 'had', 'continued', 'spending', 'the', 'whole', 'country', 'would', 'be', 'in', 'the', 'same', 'financial', 'mess', 'birmingham', 'is', 'in'] 

Lemma. without stopwords:
 ['instead', 'making', 'saving', 'continued', 'spend', 'money', 'yes', 'sound', 'responsible', 'maybe', 'government', 'continued', 'spending', 'whole', 'country', 'would', 'financial', 'mess', 'birmingham'] 

Stemmer with stopwords:
 ['so', 'instead', 'of', 'make', 'save', 'they', 'continu', 'to', 'spend', 'money', 'they', 'didnt', 'have', 'ye', 'that', 'sound',

In [16]:
class_distribution(df_reddit_sentiment_uk) # Imbalanced data 

   Count  Percentage
0    892       88.58
1    115       11.42


### BESSTIE-reddit-sentiment-au/

In [23]:
import pandas as pd

splits = {'train': 'reddit-sentiment-au-train.jsonl', 'validation': 'reddit-sentiment-au-valid.jsonl'}
df_reddit_sentiment_au = pd.read_json("hf://datasets/mindhunter23/BESSTIE-reddit-sentiment-au/" + splits["train"], lines=True)
df_reddit_sentiment_au

,id,text,sentiment_label
0,1d2d56d,"No its more about risk management, why accept ...",1
1,1d2cfsd,I don’t play this game. \n\nThem: “What are yo...,0
2,1cw9vcr,Well I'm not really confident that we'll see m...,0
3,1czvemb,He's not wrong though alot of media is RW sla...,0
4,1d3x6bo,Please contact safe transport Victoria. This i...,1
...,...,...,...
1758,1d3yzbw,I just went through this with my tenant who wa...,1
1759,1d3yn9l,Yeah my rental has a shitty little 1kw split s...,0
1760,1d0upb1,Lol like what ? go back in time and buy 150mil...,0
1761,1d639g5,"From my point of view, there is absolutely not...",0


In [4]:
class_distribution(df_reddit_sentiment_au)

                 Count  Percentage
sentiment_label                   
0                 1200       68.07
1                  563       31.93


### BESSTIE-reddit-sarcasm-in

In [5]:
import pandas as pd

splits = {'train': 'reddit-sarcasm-in-train.jsonl', 'validation': 'reddit-sarcasm-in-valid.jsonl'}
df_reddit_sarcasm_in = pd.read_json("hf://datasets/mindhunter23/BESSTIE-reddit-sarcasm-in/" + splits["train"], lines=True)
df_reddit_sarcasm_in

,id,text,sarcasm_label
0,1d2g0fr,"Sigh, this is a typical “uncle” comment. I was...",0
1,1d4xbpu,Trailer toh accha hi dikh raha tha,0
2,1d5sp5g,Dude I knew people driving the small maruti in...,0
3,1d058fa,It also gets crowded in monsoon.\nEven though ...,0
4,1d68jon,![gif](giphy|3o6gDWzmAzrpi5DQU8)\n\nMeanwhile ...,1
...,...,...,...
1681,1d11y5n,Climate change is mostly caused by rich biatch...,1
1682,1d68jon,Kuch bhi bakwas,0
1683,1d5pfy4,Akshay Kumar and anyone less than 20 years you...,1
1684,1d69xga,"CPI(M) called, they want you to be their brand...",1


In [6]:
class_distribution(df_reddit_sarcasm_in)

               Count  Percentage
sarcasm_label                   
0               1462       86.71
1                224       13.29


### BESSTIE-google-sentiment-uk

In [8]:
import pandas as pd

splits = {'train': 'google-sentiment-uk-train.jsonl', 'validation': 'google-sentiment-uk-valid.jsonl'}
df_google_sentiment_uk = pd.read_json("hf://datasets/mindhunter23/BESSTIE-google-sentiment-uk/" + splits["train"], lines=True)
df_google_sentiment_uk

,id,text,sentiment_label
0,1.046000e+20,Tricky me because I was checking in over midni...,1
1,1.161344e+20,It's lots more cheaper than the Odeon although...,1
2,1.034757e+20,My first time and last time in this place. It ...,0
3,1.073389e+20,"You know, its not bad at all, you get plenty o...",1
4,1.172204e+20,It's. It's OK for a quick fix of junk food. Re...,0
...,...,...,...
1812,1.156926e+20,Great service by the staff! The food was good ...,1
1813,1.087021e+20,Food was delicious chicken bacon avocado in br...,1
1814,1.176660e+20,It was a very nice moon display with some love...,1
1815,1.114779e+20,The food was nice but the service from one sta...,1


In [9]:
class_distribution(df_google_sentiment_uk)

                 Count  Percentage
sentiment_label                   
1                 1359       74.79
0                  458       25.21


### BESSTIE-google-sentiment-au

In [10]:
import pandas as pd

splits = {'train': 'data/google-sentiment-au-train.jsonl', 'validation': 'data/google-sentiment-au-valid.jsonl'}
df_google_sentiment_au = pd.read_json("hf://datasets/mindhunter23/BESSTIE-google-sentiment-au/" + splits["train"], lines=True)
df_google_sentiment_au

,id,text,sentiment_label
0,1.132555e+20,This was one of the best dishes I've EVER had!...,1
1,1.101411e+20,This Mexican restaurant in Penrith is a great ...,1
2,1.103038e+20,"This was not to bad, I ordered the big pork ri...",1
3,1.107520e+20,Clean cool and a nice smaller casino to check ...,1
4,1.152390e+20,Well set out. Great areas to enjoy. Good food ...,1
...,...,...,...
941,1.087996e+20,Beautiful meals and fast cocktails. Waitress w...,1
942,1.044438e+20,With a reputation for great food and terrific ...,1
943,1.134915e+20,"Nice movie theatre, only downfall are the seat...",1
944,1.073785e+20,A beautifully styled space with the fun of a s...,1


In [11]:
class_distribution(df_google_sentiment_au)

                 Count  Percentage
sentiment_label                   
1                  695       73.47
0                  251       26.53


### BESSTIE-reddit-sentiment-in

In [12]:
import pandas as pd

splits = {'train': 'reddit-sentiment-in-train.jsonl', 'validation': 'reddit-sentiment-in-valid.jsonl'}
df_reddit_sentiment_in = pd.read_json("hf://datasets/mindhunter23/BESSTIE-reddit-sentiment-in/" + splits["train"], lines=True)
df_reddit_sentiment_in

,id,text,sentiment_label
0,1d2o00l,Zepto has a mandate that the delivery boy need...,1
1,1d5fcvf,Mujhe bhi thoda paisa do,0
2,1d04uk7,Nooo don't protest against secular freedom fig...,0
3,1d5dl6q,Har 3 mahine baad kisi bhi global celebrity ko...,0
4,1d66tng,Just because you don't find anything serious b...,0
...,...,...,...
1680,1d4p9vp,Rectal Cargo.,0
1681,1d3wg8h,I mean this is the equivalent of u go to US an...,0
1682,1czj077,what went wrong with the genes after Rajiv Gan...,0
1683,1d43657,The image says the data from 2016-2017. 8 year...,0


In [14]:
class_distribution(df_reddit_sentiment_in)

                 Count  Percentage
sentiment_label                   
0                 1256       74.54
1                  429       25.46


### BESSTIE-reddit-sarcasm-uk

In [15]:
import pandas as pd

splits = {'train': 'reddit-sarcasm-uk-train.jsonl', 'validation': 'reddit-sarcasm-uk-valid.jsonl'}
df_reddit_sarcasm_uk = pd.read_json("hf://datasets/mindhunter23/BESSTIE-reddit-sarcasm-uk/" + splits["train"], lines=True)
df_reddit_sarcasm_uk

,id,text,sarcasm_label
0,1d4qqda,14 billion litres of sewage divided by 67 mill...,0
1,1d69hhx,> A prolific thief who stole 17 tubes of Pring...,1
2,1bn9s9s,It's a flakey investment now for sure. \n\nIf ...,0
3,1d3xlr8,So basically she has been barred from standing...,1
4,1cy36vt,"Rishi, the country is fucked.\n\n*That's Labou...",0
...,...,...,...
1026,1d2y53i,I can’t remember the last year that went by wh...,0
1027,1d3a6dd,"Who would want to throw their hat in the ring,...",1
1028,1d06vir,That's a dark day for women in the UK. Her sen...,0
1029,1d5wzpn,"""With UK airports missing the June deadline fo...",0


In [16]:
class_distribution(df_reddit_sarcasm_uk)

               Count  Percentage
sarcasm_label                   
0                804       77.98
1                227       22.02


### BESSTIE-reddit-sarcasm-au

In [17]:
import pandas as pd

splits = {'train': 'reddit-sarcasm-au-train.jsonl', 'validation': 'reddit-sarcasm-au-valid.jsonl'}
df_reddit_sarcasm_au = pd.read_json("hf://datasets/mindhunter23/BESSTIE-reddit-sarcasm-au/" + splits["train"], lines=True)
df_reddit_sarcasm_au

,id,text,sarcasm_label
0,1d5cns8,"This is very helpful, thankyou so much for you...",0
1,1d5fqvb,It's all about me. Something happens which is ...,1
2,1d1b31z,The Conversation: [‘Everybody has not won’: tr...,0
3,1cp6js4,>Hume said the force respected the public inte...,1
4,1d35ec2,"He’s the last gasp of what LNP used to be, not...",1
...,...,...,...
1758,1d1ngrk,It is always better to go for a job when you a...,0
1759,1csf15u,"Australia is basically like Saudi Arabia, but ...",1
1760,1d1s7yi,The Space Shuttle was dealing with ordinary ph...,0
1761,1cxtves,This is the government the Courier Mail wants ...,1


In [18]:
class_distribution(df_reddit_sarcasm_au)

               Count  Percentage
sarcasm_label                   
0               1022       57.97
1                741       42.03


### BESSTIE-google-sentiment-in

In [20]:
import pandas as pd

splits = {'train': 'google-sentiment-in-train.jsonl', 'validation': 'google-sentiment-in-valid.jsonl'}
df_google_sentiment_in = pd.read_json("hf://datasets/mindhunter23/BESSTIE-google-sentiment-in/" + splits["train"], lines=True)
df_google_sentiment_in

,id,text,sentiment_label
0,1.114268e+20,They have an amazing hospitality structure loc...,1
1,1.116605e+20,The attender attitude is not welcoming. Ordere...,0
2,1.134214e+20,The taste is good.. Decent staff.. But the atm...,0
3,1.055034e+20,"Wahi purani jagah, wahi purani yaadein..... Ra...",1
4,1.092858e+20,"An extremely over hyped biriyani, Definitely i...",1
...,...,...,...
1643,1.045586e+20,Location wise it's Good. Parking at road side ...,0
1644,1.051527e+20,"The experience was good, the atmosphere also g...",1
1645,1.032750e+20,I tried the laxmi special sandwich double chee...,1
1646,1.152006e+20,I love o yes but recently they were done blund...,0


In [21]:
class_distribution(df_google_sentiment_in)

                 Count  Percentage
sentiment_label                   
1                 1232       74.76
0                  416       25.24
